# Transfer Learning in Pytorch

This code uses the pre-trained ResNet-50 model as a feature extractor and adds a new classification layer on top of it. The pre-trained layers are frozen, and only the new classification layer is trained.
Here's what's happening:
- We load the pre-trained ResNet-50 model using models.resnet50(pretrained=True).
- We freeze the weights of the pre-trained layers using param.requires_grad = False.
- We add a new classification layer on top of the pre-trained model using model.fc = nn.Linear(model.fc.in_features, num_classes).
- We move the model to the device (GPU or CPU) using model.to(device).
- We define the data transforms and load the dataset (e.g., Flowers dataset).
- We create data loaders and define the optimizer and loss function.
- We train the model using the pre-trained layers as a feature extractor and the new classification layer as the trainable part.
Note that this is just an example code, and you may need to adjust the hyperparameters (e.g., learning rate, batch size, number of epochs) to achieve better results on your specific task.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms

# Define the device (GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the pre-trained ResNet-50 model
model = models.resnet50(pretrained=True)

# Freeze the weights of the pre-trained layers
for param in model.parameters():
    param.requires_grad = False

# Add a new classification layer on top of the pre-trained model
num_classes = 5  # Replace with the number of classes in your dataset
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move the model to the device (GPU or CPU)
model.to(device)

# Define the data transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load the dataset (e.g., Flowers dataset)
train_dataset = datasets.ImageFolder('flowers/train', data_transforms['train'])
val_dataset = datasets.ImageFolder('flowers/val', data_transforms['val'])

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define the optimizer and loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Train the model
for epoch in range(10):
    model.train()
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Epoch {epoch+1}, Batch {batch_idx+1}, Loss: {loss.item()}')

    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()

    accuracy = correct / len(val_loader.dataset)
    print(f'Epoch {epoch+1}, Val Loss: {val_loss / len(val_loader)}, Val Acc: {accuracy:.2f}%')